# 数据清洗与准备

在数据分析和建模的过程中，相当多的时间要用在数据准备上：加载、清理、转换以及重塑。这些工作会占到分析师时间的80%或更多。有时，存储在文件和数据库中的数据的格式不适合某个特定的任务。许多研究者都选择使用通用编程语言（如Python、Perl、R或Java）或UNIX文本处理工具（如sed或awk）对数据格式进行专门处理。幸运的是，pandas和内置的Python标准库提供了一组高级的、灵活的、快速的工具，可以让你轻松地将数据规整为想要的格式。
在本章中，我会讨论处理缺失数据、重复数据、字符串操作和其它分析数据转换的工具。下一章，我会关注于用多种方法合并、重塑数据集。

## 处理缺失值

在许多数据分析工作中，缺失数据是经常发生的。pandas的目标之一就是尽量轻松地处理缺失数据。例如，pandas对象的所有描述性统计默认都不包括缺失数据。

缺失数据在pandas中呈现的方式有些不完美，但对于大多数用户可以保证功能正常。对于数值数据，pandas使用浮点值NaN（Not a Number）表示缺失数据。我们称其为哨兵值，可以方便的检测出来：

In [4]:
import numpy as np
import pandas as pd
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

In [5]:
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [6]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

在pandas中，我们采用了R语言中的惯用法，即将缺失值表示为NA，它表示不可用not available。在统计应用中，NA数据可能是不存在的数据或者虽然存在，但是没有观察到（例如，数据采集中发生了问题）。当进行数据清洗以进行分析时，最好直接对缺失数据进行分析，以判断数据采集的问题或缺失数据可能导致的偏差。

Python内置的None值在对象数组中也可以作为NA：

In [7]:
string_data[0] = None

In [8]:
string_data

0         None
1    artichoke
2          NaN
3      avocado
dtype: object

In [9]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

pandas项目中还在不断优化内部细节以更好处理缺失数据，像用户API功能，例如pandas.isnull，去除了许多恼人的细节。表7-1列出了一些关于缺失数据处理的函数。

### 过滤缺失值

过滤掉缺失数据的办法有很多种。你可以通过pandas.isnull或布尔索引的手工方法，但dropna可能会更实用一些。对于一个Series，dropna返回一个仅含非空数据和索引值的Series：

In [10]:
from numpy import nan as NA

In [11]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [12]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

这等价于：

In [13]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

而对于DataFrame对象，事情就有点复杂了。你可能希望丢弃全NA或含有NA的行或列。dropna默认丢弃任何含有缺失值的行：

In [14]:
 data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
   ....:                      [NA, NA, NA], [NA, 6.5, 3.]])

In [15]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [16]:
cleaned = data.dropna()

In [17]:
cleaned

,0,1,2
0,1.0,6.5,3.0


传入how='all'将只丢弃全为NA的那些行：

In [18]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


用这种方式丢弃列，只需传入axis=1即可：

In [20]:
data[4] = NA

In [21]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [22]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


另一个滤除DataFrame行的问题涉及时间序列数据。假设你只想留下一部分观测数据，可以用thresh参数实现此目的：

In [23]:
df = pd.DataFrame(np.random.randn(7, 3))

In [24]:
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA

In [25]:
df

,0,1,2
0,0.499050,NaN,NaN
1,-1.027487,NaN,NaN
2,0.729819,NaN,-1.136329
3,0.166462,NaN,1.405814
4,0.912389,-0.219945,0.096944
5,0.258807,0.514208,1.397413
6,1.783321,-1.183523,0.157293


In [26]:
df.dropna()

,0,1,2
4,0.912389,-0.219945,0.096944
5,0.258807,0.514208,1.397413
6,1.783321,-1.183523,0.157293


In [28]:
df.dropna(thresh=2)

,0,1,2
2,0.729819,NaN,-1.136329
3,0.166462,NaN,1.405814
4,0.912389,-0.219945,0.096944
5,0.258807,0.514208,1.397413
6,1.783321,-1.183523,0.157293


### 补全缺失值

你可能不想滤除缺失数据（有可能会丢弃跟它有关的其他数据），而是希望通过其他方式填补那些“空洞”。对于大多数情况而言，fillna方法是最主要的函数。通过一个常数调用fillna就会将缺失值替换为那个常数值：

In [29]:
df.fillna(0)

,0,1,2
0,0.499050,0.000000,0.000000
1,-1.027487,0.000000,0.000000
2,0.729819,0.000000,-1.136329
3,0.166462,0.000000,1.405814
4,0.912389,-0.219945,0.096944
5,0.258807,0.514208,1.397413
6,1.783321,-1.183523,0.157293


若是通过一个字典调用fillna，就可以实现对不同的列填充不同的值：

In [30]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,0.499050,0.500000,0.000000
1,-1.027487,0.500000,0.000000
2,0.729819,0.500000,-1.136329
3,0.166462,0.500000,1.405814
4,0.912389,-0.219945,0.096944
5,0.258807,0.514208,1.397413
6,1.783321,-1.183523,0.157293


fillna默认会返回新对象，但也可以对现有对象进行就地修改：

In [31]:
_ = df.fillna(0, inplace=True)

In [32]:
df

,0,1,2
0,0.499050,0.000000,0.000000
1,-1.027487,0.000000,0.000000
2,0.729819,0.000000,-1.136329
3,0.166462,0.000000,1.405814
4,0.912389,-0.219945,0.096944
5,0.258807,0.514208,1.397413
6,1.783321,-1.183523,0.157293


对reindexing有效的那些插值方法也可用于fillna：

In [33]:
df = pd.DataFrame(np.random.randn(6, 3))

In [35]:
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA

In [36]:
df

,0,1,2
0,-0.276929,0.623558,0.355985
1,-1.079149,-1.128023,0.165296
2,-0.447753,NaN,0.123230
3,-0.281268,NaN,0.433134
4,0.242155,NaN,NaN
5,1.122231,NaN,NaN


In [37]:
df.fillna(method='ffill')

,0,1,2
0,-0.276929,0.623558,0.355985
1,-1.079149,-1.128023,0.165296
2,-0.447753,-1.128023,0.123230
3,-0.281268,-1.128023,0.433134
4,0.242155,-1.128023,0.433134
5,1.122231,-1.128023,0.433134


In [39]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-0.276929,0.623558,0.355985
1,-1.079149,-1.128023,0.165296
2,-0.447753,-1.128023,0.123230
3,-0.281268,-1.128023,0.433134
4,0.242155,NaN,0.433134
5,1.122231,NaN,0.433134


只要有些创新，你就可以利用fillna实现许多别的功能。比如说，你可以传入Series的平均值或中位数：

In [40]:
data = pd.Series([1., NA, 3.5, NA, 7])

In [41]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

表7-2列出了fillna的参考:

## 数据转换

### 删除重复值

In [43]:
 data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
   ....:                      'k2': [1, 1, 2, 3, 3, 4, 4]})

In [44]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


DataFrame的`duplicated`方法返回一个布尔型Series，表示各行是否是重复行（前面出现过的行）：

In [45]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

还有一个与此相关的drop_duplicates方法，它会返回一个DataFrame，重复的数组会标为False：

In [46]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


这两个方法默认会判断全部列，你也可以指定部分列进行重复项判断。假设我们还有一列值，且只希望根据k1列过滤重复项：

In [47]:
data['v1'] = range(7)

In [48]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicated和drop_duplicates默认保留的是第一个出现的值组合。传入keep='last'则保留最后一个：

In [49]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 使用函数或映射进行数据转换

对于许多数据集，你可能希望根据数组、Series或DataFrame列中的值来实现转换工作。我们来看看下面这组有关肉类的数据：

In [50]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
   ....:                               'Pastrami', 'corned beef', 'Bacon',
   ....:                               'pastrami', 'honey ham', 'nova lox'],
   ....:                      'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [51]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


假设你想要添加一列表示该肉类食物来源的动物类型。我们先编写一个不同肉类到动物的映射：

In [52]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

Series的map方法可以接受一个函数或含有映射关系的字典型对象，但是这里有一个小问题，即有些肉类的首字母大写了，而另一些则没有。因此，我们还需要使用Series的str.lower方法，将各个值转换为小写：

In [53]:
lowercased = data['food'].str.lower()

In [54]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [55]:
data['animal'] = lowercased.map(meat_to_animal)

In [56]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


我们也可以传入一个能够完成全部这些工作的函数：

In [57]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

使用map是一种实现元素级转换以及其他数据清理工作的便捷方式。

### 替代值

利用fillna方法填充缺失数据可以看做值替换的一种特殊情况。前面已经看到，map可用于修改对象的数据子集，而replace则提供了一种实现该功能的更简单、更灵活的方式。我们来看看下面这个Series：

In [58]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [59]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

-999这个值可能是一个表示缺失数据的标记值。要将其替换为pandas能够理解的NA值，我们可以利用replace来产生一个新的Series（除非传入inplace=True）：

In [60]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

如果你希望一次性替换多个值，可以传入一个由待替换值组成的列表以及一个替换值：

In [61]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

要让每个值有不同的替换值，可以传递一个替换列表：

In [62]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

传入的参数也可以是字典：

In [64]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 重命名轴索引

跟Series中的值一样，轴标签也可以通过函数或映射进行转换，从而得到一个新的不同标签的对象。轴还可以被就地修改，而无需新建一个数据结构。接下来看看下面这个简单的例子：

In [65]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
   ....:                     index=['Ohio', 'Colorado', 'New York'],
   ....:                     columns=['one', 'two', 'three', 'four'])

跟Series一样，轴索引也有一个map方法：

In [66]:
transform = lambda x: x[:4].upper()

In [67]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

你可以将其赋值给index，这样就可以对DataFrame进行就地修改：

In [68]:
data.index = data.index.map(transform)

In [69]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


如果想要创建数据集的转换版（而不是修改原始数据），比较实用的方法是rename：

In [70]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


特别说明一下，rename可以结合字典型对象实现对部分轴标签的更新：

In [72]:
data.rename(index={'OHIO': 'INDIANA'},
   ....:             columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


rename可以实现复制DataFrame并对其索引和列标签进行赋值。如果希望就地修改某个数据集，传入inplace=True即可：

In [73]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)

In [74]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 离散化和分箱

为了便于分析，连续数据常常被离散化或拆分为“箱子”。假设有一组人员数据，而你希望将它们划分为不同的年龄组：

In [75]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

接下来将这些数据划分为“18到25”、“26到35”、“35到60”以及“60以上”几个面元。要实现该功能，你需要使用pandas的cut函数：

In [76]:
bins = [18, 25, 35, 60, 100]

In [78]:
cats = pd.cut(ages, bins)

In [79]:
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

pandas返回的是一个特殊的Categorical对象。结果展示了pandas.cut划分的面元。你可以将其看做一组表示面元名称的字符串。它的底层含有一个表示不同分类名称的类型数组，以及一个codes属性中的年龄数据的标签：

In [80]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [81]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

pd.value_counts(cats)是pandas.cut结果的面元计数。

跟“区间”的数学符号一样，圆括号表示开端，而方括号则表示闭端（包括）。哪边是闭端可以通过right=False进行修改：

In [82]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

你可 以通过传递一个列表或数组到labels，设置自己的面元名称：

In [83]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

如果向cut传入的是面元的数量而不是确切的面元边界，则它会根据数据的最小值和最大值计算等长面元。下面这个例子中，我们将一些均匀分布的数据分成四组：

In [84]:
data = np.random.rand(20)

In [85]:
pd.cut(data, 4, precision=2)

[(0.038, 0.25], (0.25, 0.46], (0.67, 0.88], (0.67, 0.88], (0.46, 0.67], ..., (0.46, 0.67], (0.67, 0.88], (0.67, 0.88], (0.67, 0.88], (0.67, 0.88]]
Length: 20
Categories (4, interval[float64]): [(0.038, 0.25] < (0.25, 0.46] < (0.46, 0.67] < (0.67, 0.88]]

选项precision=2，限定小数只有两位。

qcut是一个非常类似于cut的函数，它可以根据样本分位数对数据进行面元划分。根据数据的分布情况，cut可能无法使各个面元中含有相同数量的数据点。而qcut由于使用的是样本分位数，因此可以得到大小基本相等的面元：

In [86]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats

[(0.0279, 0.649], (0.0279, 0.649], (-0.655, 0.0279], (-3.319, -0.655], (-0.655, 0.0279], ..., (0.0279, 0.649], (-3.319, -0.655], (-0.655, 0.0279], (0.649, 2.832], (0.0279, 0.649]]
Length: 1000
Categories (4, interval[float64]): [(-3.319, -0.655] < (-0.655, 0.0279] < (0.0279, 0.649] < (0.649, 2.832]]

In [87]:
pd.value_counts(cats)

(0.649, 2.832]      250
(0.0279, 0.649]     250
(-0.655, 0.0279]    250
(-3.319, -0.655]    250
dtype: int64

与cut类似，你也可以传递自定义的分位数（0到1之间的数值，包含端点）：

In [88]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(0.0279, 1.241], (0.0279, 1.241], (-1.352, 0.0279], (-1.352, 0.0279], (-1.352, 0.0279], ..., (0.0279, 1.241], (-3.319, -1.352], (-1.352, 0.0279], (0.0279, 1.241], (0.0279, 1.241]]
Length: 1000
Categories (4, interval[float64]): [(-3.319, -1.352] < (-1.352, 0.0279] < (0.0279, 1.241] < (1.241, 2.832]]

### 检测和过滤异常值

过滤或变换异常值（outlier）在很大程度上就是运用数组运算。来看一个含有正态分布数据的DataFrame：

In [89]:
data = pd.DataFrame(np.random.randn(1000, 4))

In [90]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.013935,-0.019973,-0.019892,0.001901
std,0.981690,1.022043,0.963110,0.993573
min,-3.173471,-3.645205,-3.374467,-3.042342
25%,-0.671233,-0.714679,-0.684571,-0.697739
50%,0.004499,-0.021572,-0.002601,0.020065
75%,0.673248,0.701548,0.625868,0.677679
max,3.110471,3.396417,3.020112,3.186593


假设你想要找出某列中绝对值大小超过3的值：

In [91]:
col = data[2]

In [92]:
col[np.abs(col) > 3]

98    -3.374467
201    3.020112
Name: 2, dtype: float64

要选出全部含有“超过3或－3的值”的行，你可以在布尔型DataFrame中使用any方法：

In [93]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
1,-0.509088,-3.645205,-0.742423,-2.285943
98,1.376757,0.446484,-3.374467,0.683270
121,-3.141490,-1.080195,1.376524,-1.504220
125,-0.508266,0.307075,1.514164,3.118016
131,-0.295330,-0.447364,-1.469811,3.186593
201,0.672831,-0.568224,3.020112,-0.652504
248,-0.177676,-3.367673,-0.143789,-1.071621
249,-3.173471,-1.483149,-0.355845,-0.198185
363,0.457486,-0.061576,0.043496,-3.042342
564,-0.357075,-3.355035,-0.036157,0.020268


根据这些条件，就可以对值进行设置。下面的代码可以将值限制在区间－3到3以内：

In [94]:
data[np.abs(data) > 3] = np.sign(data) * 3

In [95]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.014338,-0.019001,-0.019537,0.001538
std,0.979729,1.016454,0.961813,0.992191
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.671233,-0.714679,-0.684571,-0.697739
50%,0.004499,-0.021572,-0.002601,0.020065
75%,0.673248,0.701548,0.625868,0.677679
max,3.000000,3.000000,3.000000,3.000000


根据数据的值是正还是负，np.sign(data)可以生成1和-1：

In [96]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,-1.0,-1.0
1,-1.0,-1.0,-1.0,-1.0
2,1.0,1.0,-1.0,1.0
3,-1.0,1.0,1.0,-1.0
4,-1.0,-1.0,1.0,1.0


### 置换和随机抽样

利用numpy.random.permutation函数可以轻松实现对Series或DataFrame的列的排列工作（permuting，随机重排序）。通过需要排列的轴的长度调用permutation，可产生一个表示新顺序的整数数组：

In [97]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)

In [98]:
sampler

array([1, 4, 2, 3, 0])

然后就可以在基于iloc的索引操作或take函数中使用该数组了：

In [99]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [100]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
4,16,17,18,19
2,8,9,10,11
3,12,13,14,15
0,0,1,2,3


如果不想用替换的方式选取随机子集，可以在Series和DataFrame上使用sample方法：

In [101]:
df.sample(n=3)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
1,4,5,6,7


要通过替换的方式产生样本（允许重复选择），可以传递replace=True到sample：

In [103]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

3    6
0    5
2   -1
3    6
0    5
4    4
4    4
1    7
4    4
0    5
dtype: int64

### 计算指标/虚拟变量

另一种常用于统计建模或机器学习的转换方式是：将分类变量（categorical variable）转换为“哑变量”或“指标矩阵”。

如果DataFrame的某一列中含有k个不同的值，则可以派生出一个k列矩阵或DataFrame（其值全为1和0）。pandas有一个get_dummies函数可以实现该功能（其实自己动手做一个也不难）。使用之前的一个DataFrame例子：

In [104]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
   .....:                    'data1': range(6)})

In [105]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


有时候，你可能想给指标DataFrame的列加上一个前缀，以便能够跟其他数据进行合并。get_dummies的prefix参数可以实现该功能：

In [106]:
dummies = pd.get_dummies(df['key'], prefix='key')

In [107]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


如果DataFrame中的某行同属于多个分类，则事情就会有点复杂。看一下MovieLens 1M数据集，14章会更深入地研究它：

In [111]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('datasets/movielens/movies.dat', sep='::', header=None, names=mnames, engine='python')

In [112]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


要为每个genre添加指标变量就需要做一些数据规整操作。首先，我们从数据集中抽取出不同的genre值：

In [114]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)

In [115]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

构建指标DataFrame的方法之一是从一个全零DataFrame开始：

In [116]:
zero_matrix = np.zeros((len(movies), len(genres)))

In [117]:
dummies = pd.DataFrame(zero_matrix, columns=genres)

现在，迭代每一部电影，并将dummies各行的条目设为1。要这么做，我们使用dummies.columns来计算每个类型的列索引：

In [118]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [119]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

然后，根据索引，使用.iloc设定值：

In [121]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

然后，和以前一样，再将其与movies合并起来：

In [122]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [123]:
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

> 笔记：对于很大的数据，用这种方式构建多成员指标变量就会变得非常慢。最好使用更低级的函数，将其写入NumPy数组，然后结果包装在DataFrame中。

一个对统计应用有用的秘诀是：结合get_dummies和诸如cut之类的离散化函数：

In [124]:
np.random.seed(12345)
values = np.random.rand(10)

In [125]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

   (0.0, 0.2]  (0.2, 0.4]  (0.4, 0.6]  (0.6, 0.8]  (0.8, 1.0]
0           0           0           0           0           1
1           0           1           0           0           0
2           1           0           0           0           0
3           0           1           0           0           0
4           0           0           1           0           0
5           0           0           1           0           0
6           0           0           0           0           1
7           0           0           0           1           0
8           0           0           0           1           0
9           0           0           0           1           0

## 字符串操作

Python能够成为流行的数据处理语言，部分原因是其简单易用的字符串和文本处理功能。大部分文本运算都直接做成了字符串对象的内置方法。对于更为复杂的模式匹配和文本操作，则可能需要用到正则表达式。pandas对此进行了加强，它使你能够对整组数据应用字符串表达式和正则表达式，而且能处理烦人的缺失数据。

### 字符串对象方法

对于许多字符串处理和脚本应用，内置的字符串方法已经能够满足要求了。例如，以逗号分隔的字符串可以用split拆分成数段：

In [126]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

split常常与strip一起使用，以去除空白符（包括换行符）：

In [127]:
pieces = [x.strip() for x in val.split(',')]

In [128]:
pieces

['a', 'b', 'guido']

利用加法，可以将这些子字符串以双冒号分隔符的形式连接起来：

In [129]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

但这种方式并不是很实用。一种更快更符合Python风格的方式是，向字符串"::"的join方法传入一个列表或元组：

In [130]:
'::'.join(pieces)

'a::b::guido'

其它方法关注的是子串定位。检测子串的最佳方式是利用Python的in关键字，还可以使用index和find：

In [131]:
'guido' in val

True

In [132]:
val.index(',')

1

In [133]:
val.find(':')

-1

注意find和index的区别：如果找不到字符串，index将会引发一个异常（而不是返回－1）：

In [134]:
val.index(':')

ValueError: substring not found

与此相关，count可以返回指定子串的出现次数：

In [135]:
val.count(',')

2

replace用于将指定模式替换为另一个模式。通过传入空字符串，它也常常用于删除模式：

In [136]:
val.replace(',', '::')

'a::b::  guido'

In [137]:
val.replace(',', '')

'ab  guido'

表7-3列出了Python内置的字符串方法。这些运算大部分都能使用正则表达式实现。

### 正则表达式

正则表达式提供了一种灵活的在文本中搜索或匹配（通常比前者复杂）字符串模式的方式。正则表达式，常称作regex，是根据正则表达式语言编写的字符串。Python内置的re模块负责对字符串应用正则表达式。我将通过一些例子说明其使用方法。
re模块的函数可以分为三个大类：模式匹配、替换以及拆分。当然，它们之间是相辅相成的。一个regex描述了需要在文本中定位的一个模式，它可以用于许多目的。我们先来看一个简单的例子：假设我想要拆分一个字符串，分隔符为数量不定的一组空白符（制表符、空格、换行符等）。描述一个或多个空白符的regex是\s+：

In [139]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

调用re.split('\s+',text)时，正则表达式会先被编译，然后再在text上调用其split方法。你可以用re.compile自己编译regex以得到一个可重用的regex对象："

In [140]:
regex = re.compile('\s+')

In [142]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

如果只希望得到匹配regex的所有模式，则可以使用findall方法：

In [143]:
regex.findall(text)

['    ', '\t ', '  \t']

> 笔记：如果想避免正则表达式中不需要的转义（\），则可以使用原始字符串字面量如r'C:\x'（也可以编写其等价式'C:\x'）。

如果打算对许多字符串应用同一条正则表达式，强烈建议通过re.compile创建regex对象。这样将可以节省大量的CPU时间。

`match`和`search`跟`findall`功能类似。`findall`返回的是字符串中所有的匹配项，而`search`则只返回第一个匹配项。`match`更加严格，它只匹配字符串的首部。来看一个小例子，假设我们有一段文本以及一条能够识别大部分电子邮件地址的正则表达式：

In [144]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

对text使用findall将得到一组电子邮件地址：

In [145]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

search返回的是文本中第一个电子邮件地址（以特殊的匹配项对象形式返回）。对于上面那个regex，匹配项对象只能告诉我们模式在原字符串中的起始和结束位置：

In [146]:
m = regex.search(text)

In [147]:
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [148]:
text[m.start():m.end()]

'dave@google.com'

regex.match则将返回None，因为它只匹配出现在字符串开头的模式：

In [149]:
print(regex.match(text))

None


相关的，sub方法可以将匹配到的模式替换为指定字符串，并返回所得到的新字符串：

In [150]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



假设你不仅想要找出电子邮件地址，还想将各个地址分成3个部分：用户名、域名以及域后缀。要实现此功能，只需将待分段的模式的各部分用圆括号包起来即可：

In [151]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [152]:
regex = re.compile(pattern, flags=re.IGNORECASE)

由这种修改过的正则表达式所产生的匹配项对象，可以通过其groups方法返回一个由模式各段组成的元组：

In [153]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

对于带有分组功能的模式，findall会返回一个元组列表：

In [155]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

sub还能通过诸如\1、\2之类的特殊符号访问各匹配项中的分组。符号\1对应第一个匹配的组，\2对应第二个匹配的组，以此类推：

In [156]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



Python中还有许多的正则表达式，但大部分都超出了本书的范围。表7-4是一个简要概括。

### pandas中的向量化字符串函数

清理待分析的散乱数据时，常常需要做一些字符串规整化工作。更为复杂的情况是，含有字符串的列有时还含有缺失数据：

In [157]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
   .....:         'Rob': 'rob@gmail.com', 'Wes': np.nan}

In [159]:
data = pd.Series(data)

In [160]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [161]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

通过data.map，所有字符串和正则表达式方法都能被应用于（传入lambda表达式或其他函数）各个值，但是如果存在NA（null）就会报错。为了解决这个问题，Series有一些能够跳过NA值的面向数组方法，进行字符串操作。通过Series的str属性即可访问这些方法。例如，我们可以通过str.contains检查各个电子邮件地址是否含有"gmail"：

In [162]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

也可以使用正则表达式，还可以加上任意re选项（如IGNORECASE）：

In [163]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [164]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

有两个办法可以实现矢量化的元素获取操作：要么使用str.get，要么在str属性上使用索引：

In [165]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

要访问嵌入列表中的元素，我们可以传递索引到这两个函数中：

In [166]:
matches.str.get(1)

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [167]:
matches.str[0]

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

你可以利用这种方法对字符串进行截取：

In [168]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

表7-5介绍了更多的pandas字符串方法。